In [11]:
import string
import random
import pandas as pd
from typing import List
from pyspark.sql.pandas.functions import pandas_udf
from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
%load_ext autoreload
%autoreload 2
%matplotlib inline

COLUMNS = ["name","age","salary"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


比较下开不开启arrow的区别

测试下来，感觉性能提升有点奇怪。有时候会快，有时候会慢。

In [2]:
def create_random_data(row_num:int)->List[tuple]:
     result = list()
     a_str = string.ascii_uppercase
     for i in range(row_num):
         random_letter = random.choice(a_str)
         result.append((random_letter,random.randint(1,row_num),random.random()))
     return result

In [6]:
def test_performance(session_factory:DemoSQLSessionFactory, n:int = 100000):
    data = create_random_data(n)

    spark_session = session_factory.build_session()

    df = spark_session.createDataFrame(data,COLUMNS).cache()
    df.toPandas().head(5)

2.9 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
session_factory_normal = DemoSQLSessionFactory(name="normal")
%timeit test_performance(session_factory_normal)

In [4]:
session_factory_arrow = DemoSQLSessionFactory(name="with arraw")
session_factory_arrow.add_config("spark.sql.execution.arrow.pyspark.enabled","true")
%timeit test_performance(session_factory_arrow)

2.72 s ± 59.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


常规的HelloWorld的example。
页面上面的第一个例子。本质就是生成一个新的dataframe
1. 在annotation上面列出的是新的dataframe的col和类型
2. 他会自动的把pd的转换成spark的

In [12]:
spark = session_factory_arrow.build_session()
test_data = create_random_data(row_num=10)
basic_df = spark.createDataFrame(test_data,COLUMNS)
basic_df.show()

+----+---+-------------------+
|name|age|             salary|
+----+---+-------------------+
|   M|  7|0.26350331394919146|
|   H| 10|0.30811575540322644|
|   K|  6| 0.4720255241377238|
|   R|  7| 0.4493333727303095|
|   P|  6| 0.7987328763787399|
|   S|  2| 0.6795708906792355|
|   N|  1| 0.9808400874816904|
|   Y|  1| 0.2891593818237852|
|   R|  8| 0.3088092627589295|
|   H|  4|0.29739758351698586|
+----+---+-------------------+



In [36]:
@pandas_udf("total double")
def func(s1: pd.Series, s2: pd.Series) -> pd.DataFrame:
    s3 = pd.DataFrame()
    s3['total'] = s1 + s2
    return s3
basic_df.select(func("age","salary").alias("result")).show()


+--------------------+
|              result|
+--------------------+
| [7.263503313949191]|
|[10.308115755403227]|
| [6.472025524137724]|
|  [7.44933337273031]|
|  [6.79873287637874]|
|[2.6795708906792353]|
|[1.9808400874816905]|
|[1.2891593818237852]|
|  [8.30880926275893]|
| [4.297397583516986]|
+--------------------+

